<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/158_CrewAI_Pricing_Compliance_Agent_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 🧠 CrewAI Mental Model

CrewAI works like a **collaborative task force** where:

* Each **agent** has a **role**, **goal**, and **tools**
* A **coordinator** (the "crew") assigns tasks to agents
* Agents operate **sequentially**, passing their outputs to the next

So instead of a graph or state dict, you express workflows as **task + role assignments**.

---

## 📓 Use Case Recap: Pricing Compliance Agent

### 👥 Agents:

* **Fetcher**: Retrieves product pricing info
* **Policy Analyst**: Performs RAG on compliance rules
* **Compliance Officer**: Evaluates if the pricing complies
* *(Optional)* **Manager**: Reviews if escalation is triggered


---

## ✅ Output

You’ll get a natural language chain of reasoning like:

```text
🧠 Fetcher: Pricing Info: Base Price: $100, Discount: 40%
🧠 Analyst: Relevant Rule: Discounts over 30% require manager approval.
🧠 Officer: This pricing violates policy. Escalation required.
```

---

## ✅ CrewAI Highlights for Orchestration

| Feature                | How it Helps                                       |
| ---------------------- | -------------------------------------------------- |
| 🧠 Role assignment     | Encourages modular thinking — each step = a role   |
| 📋 Task definition     | Natural orchestration model (context → next agent) |
| 🪢 No glue code        | Less Python, more “describe-the-workflow”          |
| 👥 Conversational flow | Feels like simulating a human compliance team      |
| ❌ No branching logic   | Less control compared to LangGraph routing         |

---

## 🧭 Summary: CrewAI vs LangGraph vs LangChain

| Feature             | LangChain    | LangGraph                 | CrewAI                       |
| ------------------- | ------------ | ------------------------- | ---------------------------- |
| Orchestration Style | Manual       | Graph/state machine       | Role + task-based            |
| Control Flow        | Custom logic | Conditional edges         | Sequential                   |
| Reusability         | ✅            | ✅✅                        | ✅ (via roles)                |
| Conversational      | ❌            | ❌                         | ✅✅✅                          |
| Best For            | Prototyping  | Complex flows, governance | Human-style workflows, teams |



## 🚀 CrewAI Implementation

In [ ]:
# ✨ Install dependencies
# !pip install crewai openai faiss-cpu langchain

# 📦 Imports
from crewai import Agent, Task, Crew
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# ============================
# ⚙️ Set up tools (RAG utility for reuse)
def retrieve_compliance_rules(pricing_info: str) -> str:
    compliance_docs = [
        "Discounts over 30% require manager approval.",
        "All pricing must be auditable and linked to policy ID."
    ]
    vectorstore = FAISS.from_texts(compliance_docs, OpenAIEmbeddings())
    retriever = vectorstore.as_retriever()
    docs = retriever.get_relevant_documents(pricing_info)
    return "\n".join([doc.page_content for doc in docs])

# ============================
# 🧠 Define Agents

fetcher = Agent(
    role="Data Fetcher",
    goal="Provide accurate pricing information for a given product",
    backstory="An internal pricing bot trained on product data",
    verbose=True,
    allow_delegation=False,
    llm=OpenAI(temperature=0),
)

analyst = Agent(
    role="Policy Analyst",
    goal="Identify relevant compliance rules for the given pricing",
    backstory="An expert in pricing policy and corporate compliance documents",
    verbose=True,
    allow_delegation=False,
    llm=OpenAI(temperature=0),
)

officer = Agent(
    role="Compliance Officer",
    goal="Determine if pricing violates any rules and decide if escalation is needed",
    backstory="Senior compliance expert responsible for flagging violations",
    verbose=True,
    allow_delegation=False,
    llm=OpenAI(temperature=0),
)

# ============================
# 🎯 Define Tasks

fetch_task = Task(
    description="Fetch pricing for Product 123. Include base price and discount.",
    expected_output="Pricing Info: Base Price: $100, Discount: 40%",
    agent=fetcher
)

analyze_task = Task(
    description=(
        "Given the pricing info, use RAG over compliance rules to extract relevant policies. "
        "Be concise and list any rules that may apply to discounting or auditability."
    ),
    context=[fetch_task],
    agent=analyst
)

evaluate_task = Task(
    description=(
        "Based on the pricing and compliance rules, determine if the pricing complies. "
        "If discount exceeds limits, escalate to manager. Otherwise, approve."
    ),
    context=[analyze_task],
    agent=officer
)

# ============================
# 👥 Assemble the Crew

crew = Crew(
    agents=[fetcher, analyst, officer],
    tasks=[fetch_task, analyze_task, evaluate_task],
    verbose=True
)

# ============================
# 🚀 Run the Crew
results = crew.kickoff()
print("\n🔍 Final Result:\n", results)



## 🧩 Quick Recap: What’s in Your CrewAI Agent

Your **CrewAI version** builds a multi-agent team with 3 roles:

1. **Fetcher** – gets pricing info
2. **Analyst** – uses RAG to retrieve relevant compliance rules
3. **Officer** – decides whether to approve or escalate based on policy

The workflow is defined as:

* Agents: `Agent(...)`
* Tasks: `Task(...)`
* Execution: `crew.kickoff()` runs them in sequence.

---

## 🧠 Comparing CrewAI to Your Agent Recipe

Let’s evaluate how well **CrewAI** aligns with the principles you laid out in your **Agent Recipe**.

| 🧪 Design Principle                      | ✅ CrewAI Support? | Commentary                                                                                                      |
| ---------------------------------------- | ----------------- | --------------------------------------------------------------------------------------------------------------- |
| **1. Modular, functional design**        | ✅ *Mostly*        | Agents and tasks are modular classes — easy to isolate and reuse                                                |
| **2. Avoid tight coupling**              | ⚠️ *Partially*    | Tasks are chained by context (e.g., `context=[fetch_task]`), which creates implicit dependencies                |
| **3. Auditable / Inspectable Workflow**  | ⚠️ *Minimal*      | You get final output via `results`, but internal agent state isn't exposed like LangGraph's `AgentState`        |
| **4. Reusable components (nodes/tools)** | ✅ *Yes*           | You can reuse agents and tools, though not as composable as LangGraph functions                                 |
| **5. Statefulness across steps**         | ❌ *Very limited*  | There’s no centralized state object — context flows between tasks, but isn’t globally modifiable or inspectable |
| **6. DAG-style orchestration & routing** | ❌ *Not supported* | CrewAI executes linearly in task order — no branching or routing logic                                          |
| **7. Tool + LLM blending in tasks**      | ✅ *Good fit*      | The `retrieve_compliance_rules()` function integrates a FAISS retriever inside an agent pipeline — very clean   |

---

## 🧠 Key Strengths of CrewAI

* ✅ **High-level abstraction**: You can define agents and tasks with natural-language goals — great for prototyping quickly.
* ✅ **Automatic sequencing**: Just list tasks and assign agents — execution order flows top-down.
* ✅ **LLM-native focus**: Everything revolves around LLMs doing reasoning or retrieval.

---

## ⚠️ Key Limitations vs LangGraph

| Feature                          | CrewAI                      | LangGraph                                       |
| -------------------------------- | --------------------------- | ----------------------------------------------- |
| **Global state object**          | ❌ None                      | ✅ `AgentState` with full traceability           |
| **Custom routing/logic**         | ❌ Not possible              | ✅ Conditional branches (if/else, looping, etc.) |
| **Node visualization/debugging** | ❌ No native graph           | ✅ Built-in Mermaid + state at each node         |
| **Control flow transparency**    | ⚠️ Implicit (via context)   | ✅ Explicit (via graph edges)                    |
| **Reusability of logic**         | ⚠️ Bound to `Agent` classes | ✅ Fully decoupled functions or chains           |

---

## ✨ Takeaway Summary

| Platform      | Best When You Want…                                                                               |
| ------------- | ------------------------------------------------------------------------------------------------- |
| **LangGraph** | Full control, auditability, logic routing, debugging — like building Airflow pipelines for agents |
| **CrewAI**    | Quick orchestration of LLM-based collaborators with minimal boilerplate — fast and lightweight    |

---

## 👁 Final Thought

Your **Agent Recipe** — focused on modularity, low coupling, reusability, and auditability — aligns **much more closely with LangGraph** than CrewAI.

CrewAI still has a place — for **fast prototyping**, **simple workflows**, or **LLM-heavy reasoning agents** where branching/state isn’t a concern.






## 🥊 LangGraph vs. CrewAI: Who Are They Built For?

| Comparison Dimension        | **LangGraph** 🧠                                                        | **CrewAI** 🧩                                                        |
| --------------------------- | ----------------------------------------------------------------------- | -------------------------------------------------------------------- |
| **Target Audience**         | Developers, engineers, infra-minded data scientists                     | Low-code / no-code builders, LLM tinkerers                           |
| **Control Level**           | ✨ Maximum — full control over data, state, flow, routing                | ⚙️ Medium — abstracted control via agent-task pairing                |
| **Mental Model**            | Like designing an Airflow or Ray pipeline with strict state propagation | Like assigning roles in a play and letting actors ad-lib             |
| **Customization**           | Deep — tools, APIs, logic, memory, routing, everything pluggable        | Shallow — agent behavior controlled mostly via prompt context        |
| **Debuggability**           | ✅ High — you can trace, test, and simulate any node                     | ⚠️ Low — debugging internal decisions is harder without verbose logs |
| **Visual Workflow Support** | 🧩 DAG, nodes, Mermaid, edges, conditional branches                     | ❌ Linear task chain — no visual graph or node routing                |
| **Reusability**             | ✅ High — each node is just a Python function or LangChain chain         | ⚠️ Medium — agents/tasks are reusable but not composable             |
| **State Awareness**         | ✅ Core to the framework (via `AgentState`)                              | ❌ Minimal — context gets passed between tasks, but no unified state  |

---

## 🧠 In Plain English

* **LangGraph** is like building an **LLM-powered backend microservice**:
  You’re explicitly wiring nodes, logic, and state — like a pro data engineer or ML ops dev would.

* **CrewAI** is like assigning **roles to AI teammates in a Google Doc**, then asking them to work together:

  * You describe their job (via prompt).
  * Link them via tasks.
  * Hit "Go."

No wiring, no state schema, no routing — just narrative flow.

---

## 🚀 TL;DR

> **Yes — CrewAI is better for non-coders or quick iteration, while LangGraph is for professionals who care about structure, testing, observability, and control.**

CrewAI = No-code AirTable-style collaboration
LangGraph = DAG + state machine + orchestration framework for agents

